## This script is to test the results of FBA and pFBA and compare them with COBRA Toolbox
- Models used: E. coli iAF1260 (Biomodels)
- iYli21
- iMT1026v3

In [2]:
# imports
import cobra
import yaml
import pandas
from cobra.flux_analysis import pfba
from cobra.medium import minimal_medium


import sys # append path

sys.path.append('../scripts/')
import helperFunction as hf

### If you want to reload the hf without closing the file or vs code: 
# from importlib import reload
# reload(hf)

# config
config_name = 'model_config'
config_path = f'../config/{config_name}.yaml'

# load config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

#### E. coli

In [12]:
# load models
ecoli_model = cobra.io.read_sbml_model(config['models']['ecoli'])
ecoli_model.summary() #  normal fba: 0.7367 (glc__D_e uptake: 8)

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.00349,0,0.00%
cl_e,EX_cl_e,0.00349,0,0.00%
cobalt2_e,EX_cobalt2_e,0.002327,0,0.00%
cu2_e,EX_cu2_e,0.002327,0,0.00%
fe2_e,EX_fe2_e,0.0108,0,0.00%
glc__D_e,EX_glc__D_e,8,6,100.00%
k_e,EX_k_e,0.1308,0,0.00%
mg2_e,EX_mg2_e,0.005816,0,0.00%
mn2_e,EX_mn2_e,0.002327,0,0.00%
mobd_e,EX_mobd_e,0.002327,0,0.00%


In [13]:
# Run FBA and pFBA (=> same results as in COBRAToolbox)
ecoli_sol = ecoli_model.optimize()
print(ecoli_sol.objective_value) # 0.7367
biomass_id = 'BIOMASS_Ec_iAF1260_core_59p81M' 
ecoli_pfba = pfba(ecoli_model)
ecoli_pfba[biomass_id] # 0.7367

0.7367009388648788


0.7367009388648791

In [14]:
# Run FVA (40s)
ecoli_fva = cobra.flux_analysis.flux_variability_analysis(ecoli_model)

In [27]:
# get FVA result of the reaction ENO
ecoli_fva.loc['ENO',:] # 11.8032
#           COBRApy:   COBRAToolbox
# minimum: 11.804143 / 11.8032
# maximum: 11.804143 / 11.8045


minimum    11.804143
maximum    11.804143
Name: ENO, dtype: float64

In [24]:
# check how to read the ec code of a reaction if given as rdf resource
for rxn in ecoli_model.reactions:
    if 'ec-code' in rxn.annotation.keys():
        if rxn.annotation['ec-code'] == '4.2.1.11':
            print(rxn.name, rxn.id)



Enolase ENO


#### iYali4

In [32]:
# load model
yali4_corr = cobra.io.read_sbml_model(config['models']['yli4_corr'])
yali4_corr.summary() # 1.377 (same as COBRA Toolbox)

In [34]:
# Run FBA and pFBA (=> same results as in COBRAToolbox)
yali4_sol = yali4_corr.optimize()
print(yali4_sol.objective_value) # 1.37
biomass_id = 'biomass_C' 
yali4_pfba = pfba(yali4_corr)
yali4_pfba[biomass_id] # 1.37

1.3775770431148806


1.377577043114881

In [37]:
for rxn in yali4_corr.reactions:
    if 'enolase' in rxn.name.lower():
        print(rxn.name, rxn.id) 

enolase 366


In [36]:
# Run FVA (40s)
yali4_corr_fva = cobra.flux_analysis.flux_variability_analysis(yali4_corr)

In [39]:
yali4_corr_fva.loc['366',:] # enolase 12.41 (same as COBRA Toolbox)

minimum    12.411974
maximum    12.411974
Name: 366, dtype: float64

#### iMT1026v3

In [2]:
# load model
iMT1026v3_model = cobra.io.read_sbml_model(config['models']['ppa1026v3'])


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_oligoP_v "oligophosphate (5-P)">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_polyP_v "polyphosphate (100-P)">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_h2o_v "H2O">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_h2o_c "H2O">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_ppi_c "Diphosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_h_c "H+">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_pi_c

In [43]:
iMT1026v3_model.summary() # 0.057

Metabolite,Reaction,Flux,C-Number,C-Flux
glyc_e,Ex_glyc,1,3,100.00%
nh4_e,Ex_nh4,0.3582,0,0.00%
o2_e,Ex_o2,1.45,0,0.00%
pi_e,Ex_pi,0.01453,0,0.00%
so4_e,Ex_so4,0.002533,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
biomass_e,Ex_biomass,-0.05775,0,0.00%
co2_e,Ex_co2,-0.9782,1,100.00%
h_e,Ex_h,-0.3754,0,0.00%
h2o_e,Ex_h2o,-2.715,0,0.00%


In [46]:
# Run FBA and pFBA (=> same results as in COBRAToolbox)
iMT1026v3_sol = iMT1026v3_model.optimize()
print(iMT1026v3_sol.objective_value) # 0.057
biomass_id = 'Ex_biomass' 
iMT1026v3_pfba = pfba(iMT1026v3_model)
iMT1026v3_pfba[biomass_id] # 0.057

0.057750040147866416


0.05775004014786642

In [47]:
for rxn in iMT1026v3_model.reactions:
    if 'enolase' in rxn.name.lower():
        print(rxn.name, rxn.id) # ENO

enolase ENO


In [48]:
iMT1026v3_fva = cobra.flux_analysis.flux_variability_analysis(iMT1026v3_model)

In [49]:
iMT1026v3_fva.loc['ENO',:] #  0.653 same as COBRA Toolbox

minimum    0.653423
maximum    0.653433
Name: ENO, dtype: float64

### iLC915

In [3]:
# load model
iLC915_model = cobra.io.read_sbml_model(config['models']['ppaiLC915'])
####
# biomass: r1133
# glucose: r1145
# o2: r1160
# co2: r1137
# glycerol: r1148
# methanol: r1158

# # Set the objective function
# iLC915_model.objective = 'r1133'

# # Set the carbon source to glucose
# glucose_exchange = iLC915_model.reactions.get_by_id('glucose_exchange_reaction')
# glucose_exchange.lower_bound = -10  # Assuming a glucose uptake rate of -10 mmol/gDW/hr

# Perform normal FBA
solution_fba = iLC915_model.optimize()

# # Perform pFBA
# iLC915_model_pfba = iLC915_model.copy()
# iLC915_model_pfba.objective = iLC915_model_pfba.reactions.get_by_id('objective_reaction_id')
# solution_pfba = cobra.flux_analysis.pfba(iLC915_model_pfba)

# Display the solutions
print('Normal FBA Solution:')
print(solution_fba)

# print('pFBA Solution:')
# print(solution_pfba)


Normal FBA Solution:
<Solution 79.021 at 0x20e3be53760>
